In [4]:
from requests_html import HTMLSession


In [7]:
from requests_html import AsyncHTMLSession
import asyncio

# Patch the event loop for Jupyter Notebooks or similar environments
import nest_asyncio
nest_asyncio.apply()

async def fetch_and_render(url):
    session = AsyncHTMLSession()
    response = await session.get(url)
    
    # Render the JavaScript on the page. You might need to adjust the sleep time
    await response.html.arender(sleep=1)  # `sleep` time may need to be adjusted based on the page's JavaScript
    
    # Now you can access the rendered HTML
    print(response.html.html)  # or perform your extraction logic here
    
    await session.close()

async def main():
    url = "https://www.abstractsonline.com/pp8/#!/20272/session/162"
    await fetch_and_render(url)

# Run the main function
asyncio.run(main())



/Users/brendan/Desktop - Brendan’s MacBook Air/llm/lib/python3.10/site-packages/pyppeteer/util.py:29: RuntimeWarning: coroutine 'main' was never awaited
  gc.collect()
[INFO] Starting Chromium download.
100%|████████████████████████████████████████| 141M/141M [00:14<00:00, 9.78Mb/s]
[INFO] Beginning extraction
[INFO] Chromium extracted to: /Users/brendan/Library/Application Support/pyppeteer/local-chromium/1181205


BrowserError: Browser closed unexpectedly:


In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Specify the path to chromedriver
chromedriver_path = '/Users/brendan/Desktop - Brendan’s MacBook Air/selenium_folder/chromedriver-mac-arm64/chromedriver'

# Create a Service object passing the path to the chromedriver
service = Service(executable_path=chromedriver_path)

# Use the Service object to start the ChromeDriver
driver = webdriver.Chrome(service=service)

# Go to the website
driver.get('https://www.abstractsonline.com/pp8/#!/20272/session/162')

# Wait for the <blockquote> element to be present
wait = WebDriverWait(driver, 10)  # Wait for up to 10 seconds
blockquote_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#body > div > div:nth-child(1) > div > blockquote')))

# Once the <blockquote> is present, locate the <p> tag within it
p_element = blockquote_element.find_element(By.TAG_NAME, 'p')

# Now you can access the text of the <p> tag
p_text = p_element.text

# Here, you can do whatever you need with the text, such as printing it
print(p_text)

# Don't forget to close the browser when you're done
driver.quit()



This session aims to provide new investigators (postdoctoral fellows and new faculty) practical tools to write competitive grants. Focus is on how to write well for grants, how to structure an aims page to help the reviewer “get” all the information you want them to, and how to write an aims page that makes a reviewer excited about your grant. We will address different types of grants from NIH (Rs, Ks, Fs, etc) as well as grants from other countries. We will also discuss Summary Statements ("pink sheets") and have a panel where participants can ask experienced scientists about grants and grant review.


In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# Specify the path to chromedriver
chromedriver_path = '/Users/brendan/Desktop - Brendan’s MacBook Air/selenium_folder/chromedriver-mac-arm64/chromedriver'

# Create a Service object passing the path to the chromedriver
service = Service(executable_path=chromedriver_path)

# Use the Service object to start the ChromeDriver
driver = webdriver.Chrome(service=service)

# Go to the website
driver.get('https://www.abstractsonline.com/pp8/#!/20272/session/162')

# Wait for the session title element to be present
wait = WebDriverWait(driver, 10)  # Wait for up to 10 seconds

# Extract the session title
session_title_element = wait.until(EC.presence_of_element_located((By.ID, 'spnSessionTitle')))
session_title = session_title_element.text

# Wait for the <blockquote> element to be present and extract the text from the <p> tag within it
blockquote_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#body > div > div:nth-child(1) > div > blockquote')))
p_element = blockquote_element.find_element(By.TAG_NAME, 'p')
p_text = p_element.text

# Data to be saved in DataFrame
data = {
    'Title': [session_title],
    'Text': [p_text]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)

# Don't forget to close the browser when you're done
driver.quit()


                                               Title  \
0  PAS01 - Grant Writing Workshop: Tips for Succe...   

                                                Text  
0  This session aims to provide new investigators...  


In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from bs4 import BeautifulSoup

# Setup WebDriver (assuming Chrome)
chromedriver_path = '/Users/brendan/Desktop - Brendan’s MacBook Air/selenium_folder/chromedriver-mac-arm64/chromedriver'
service = Service(executable_path=chromedriver_path)
driver = webdriver.Chrome(service=service)
driver.get('https://www.abstractsonline.com/pp8/#!/20272/session/162')

# Wait for the div element containing the table to load
wait = WebDriverWait(driver, 10)
div_element = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'span7.presentation-table')))

# Locate the table within the div
table = div_element.find_element(By.TAG_NAME, 'table')

# Extract rows from the table
rows = table.find_elements(By.TAG_NAME, 'tr')

# Initialize a list to hold all row data
data = []

# Loop through rows to extract information
for row in rows:
    # Initialize dictionary for row data
    row_data = {"Time": "", "Presenter": "", "Affiliation": ""}
    
    # Extract time
    time_col = row.find_element(By.CSS_SELECTOR, 'td:first-child span.range').text
    row_data["Time"] = time_col
    
    # Extract presenter and affiliation using BeautifulSoup for easier HTML parsing
    details_html = row.find_element(By.CSS_SELECTOR, 'td:nth-child(2)').get_attribute('innerHTML')
    soup = BeautifulSoup(details_html, 'html.parser')
    
    # Extract presenter name
    presenter = soup.find('i').text if soup.find('i') else ""
    row_data["Presenter"] = presenter
    
    # Extract affiliation text
    affiliation = soup.find('p', class_='participants').text.replace(presenter, '').strip('. ')
    row_data["Affiliation"] = affiliation
    
    # Append row data to the main data list if there's valid content
    if presenter:  # To ensure we don't add empty rows
        data.append(row_data)

# Create a DataFrame from the extracted data
df = pd.DataFrame(data)

# Display the DataFrame

print(data)
# Close the browser when done
driver.quit()


                 Time           Presenter  \
0   1:00 PM - 4:00 PM       Oliver Bogler   
1   1:00 PM - 4:00 PM    Mariana C. Stern   
2   1:00 PM - 4:00 PM  Tiffany A. Wallace   
3   1:00 PM - 1:05 PM     Brian M. Rivers   
4   1:05 PM - 1:15 PM    Mariana C. Stern   
5   1:15 PM - 1:30 PM  Tiffany A. Wallace   
6   1:30 PM - 1:45 PM  Lorenzo De La Rica   
7   1:45 PM - 2:00 PM       Oliver Bogler   
8   2:45 PM - 3:05 PM     Shahrooz Vahedi   
9   3:05 PM - 3:35 PM      Robert A. Winn   
10  3:50 PM - 4:00 PM       Oliver Bogler   

                                          Affiliation  
0             National Cancer Institute, Bethesda, MD  
1   USC Norris Comprehensive Cancer Center, Los An...  
2                            NCI-CRCHD, Rockville, MD  
3           Morehouse School of Medicine, Atlanta, GA  
4   USC Norris Comprehensive Cancer Center, Los An...  
5                            NCI-CRCHD, Rockville, MD  
6                 Cancer Research UK, London, Austria  
7          

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from bs4 import BeautifulSoup

# Setup WebDriver (assuming Chrome)
chromedriver_path = '/Users/brendan/Desktop - Brendan’s MacBook Air/selenium_folder/chromedriver-mac-arm64/chromedriver'
service = Service(executable_path=chromedriver_path)
driver = webdriver.Chrome(service=service)
driver.get('https://www.abstractsonline.com/pp8/#!/20272/session/162')

# Wait for the div element containing the table to load
wait = WebDriverWait(driver, 10)
div_element = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'span7.presentation-table')))

# Locate the table within the div
table = div_element.find_element(By.TAG_NAME, 'table')

# Extract rows from the table
rows = table.find_elements(By.TAG_NAME, 'tr')

# Initialize a list to hold all row data
data = []

# Loop through rows to extract information
for row in rows:
    # Initialize dictionary for row data
    row_data = {"Time": "", "Presenter": "", "Affiliation": ""}
    
    # Extract time
    time_col = row.find_element(By.CSS_SELECTOR, 'td:first-child span.range').text
    row_data["Time"] = time_col
    
    # Extract presenter and affiliation using BeautifulSoup for easier HTML parsing
    details_html = row.find_element(By.CSS_SELECTOR, 'td:nth-child(2)').get_attribute('innerHTML')
    soup = BeautifulSoup(details_html, 'html.parser')
    
    # Extract presenter name
    presenter = soup.find('i').text if soup.find('i') else ""
    row_data["Presenter"] = presenter
    
    # Extract affiliation text
    affiliation = soup.find('p', class_='participants').text.replace(presenter, '').strip('. ')
    row_data["Affiliation"] = affiliation
    
    # Append row data to the main data list if there's valid content
    if presenter:  # To ensure we don't add empty rows
        data.append(row_data)


# Display the DataFrame

print(data)
# Close the browser when done
driver.quit()


[{'Time': '1:00 PM - 4:00 PM', 'Presenter': 'Oliver Bogler', 'Affiliation': 'National Cancer Institute, Bethesda, MD'}, {'Time': '1:00 PM - 4:00 PM', 'Presenter': 'Mariana C. Stern', 'Affiliation': 'USC Norris Comprehensive Cancer Center, Los Angeles, CA'}, {'Time': '1:00 PM - 4:00 PM', 'Presenter': 'Tiffany A. Wallace', 'Affiliation': 'NCI-CRCHD, Rockville, MD'}, {'Time': '1:00 PM - 1:05 PM', 'Presenter': 'Brian M. Rivers', 'Affiliation': 'Morehouse School of Medicine, Atlanta, GA'}, {'Time': '1:05 PM - 1:15 PM', 'Presenter': 'Mariana C. Stern', 'Affiliation': 'USC Norris Comprehensive Cancer Center, Los Angeles, CA'}, {'Time': '1:15 PM - 1:30 PM', 'Presenter': 'Tiffany A. Wallace', 'Affiliation': 'NCI-CRCHD, Rockville, MD'}, {'Time': '1:30 PM - 1:45 PM', 'Presenter': 'Lorenzo De La Rica', 'Affiliation': 'Cancer Research UK, London, Austria'}, {'Time': '1:45 PM - 2:00 PM', 'Presenter': 'Oliver Bogler', 'Affiliation': 'National Cancer Institute, Bethesda, MD'}, {'Time': '2:45 PM - 3:05

In [25]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Setup WebDriver (assuming Chrome)
chromedriver_path = '/Users/brendan/Desktop - Brendan’s MacBook Air/selenium_folder/chromedriver-mac-arm64/chromedriver'
service = Service(executable_path=chromedriver_path)
driver = webdriver.Chrome(service=service)

# Initialize a dictionary to hold data for all sessions
all_sessions_data = {}

# Loop through session numbers
for session_number in range(170, 500):
    # Format the URL with the current session number
    url = f'https://www.abstractsonline.com/pp8/#!/20272/session/{session_number}'
    driver.get(url)
    
    # Attempt to extract the session title and the specific text
    try:
        wait = WebDriverWait(driver, 10)
        session_title_element = wait.until(EC.presence_of_element_located((By.ID, 'spnSessionTitle')))
        session_title = session_title_element.text
        
        blockquote_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#body > div > div:nth-child(1) > div > blockquote')))
        p_element = blockquote_element.find_element(By.TAG_NAME, 'p')
        p_text = p_element.text
    except Exception as e:
        print(f"Data not found for session {session_number}: {e}")
        continue
    
    # Initialize a list to hold row data for the current session
    presentation_data = []
    
    # Extract presentation details
    try:
        div_element = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'span7.presentation-table')))
        table = div_element.find_element(By.TAG_NAME, 'table')
        rows = table.find_elements(By.TAG_NAME, 'tr')
        
        for row in rows:
            row_data = {"Time": "", "Presenter": "", "Affiliation": ""}
            time_col = row.find_element(By.CSS_SELECTOR, 'td:first-child span.range').text
            row_data["Time"] = time_col
            
            details_html = row.find_element(By.CSS_SELECTOR, 'td:nth-child(2)').get_attribute('innerHTML')
            soup = BeautifulSoup(details_html, 'html.parser')
            presenter = soup.find('i').text if soup.find('i') else ""
            affiliation = soup.find('p', class_='participants').text.replace(presenter, '').strip('. ')
            row_data["Presenter"] = presenter
            row_data["Affiliation"] = affiliation
            
            if presenter:
                presentation_data.append(row_data)
    except Exception as e:
        print(f"Error extracting presentation data for session {session_number}: {e}")

    # Save the session title, text, and presentation data
    all_sessions_data[session_number] = {
        'Session Title': session_title,
        'Text': p_text,
        'Presentations': presentation_data
    }

# Close the browser when done
driver.quit()

# Optionally, print or process all_sessions_data as needed
#print(all_sessions_data)


Data not found for session 171: Message: 
Stacktrace:
0   chromedriver                        0x0000000104dd453c chromedriver + 3966268
1   chromedriver                        0x0000000104dccac8 chromedriver + 3934920
2   chromedriver                        0x0000000104a4fda0 chromedriver + 277920
3   chromedriver                        0x0000000104a92394 chromedriver + 549780
4   chromedriver                        0x0000000104acabf0 chromedriver + 781296
5   chromedriver                        0x0000000104a86fb0 chromedriver + 503728
6   chromedriver                        0x0000000104a87a28 chromedriver + 506408
7   chromedriver                        0x0000000104d99724 chromedriver + 3725092
8   chromedriver                        0x0000000104d9dc18 chromedriver + 3742744
9   chromedriver                        0x0000000104d8220c chromedriver + 3629580
10  chromedriver                        0x0000000104d9e714 chromedriver + 3745556
11  chromedriver                        0x0000000

In [34]:
import json

# Specify the filename you want to save the data to
filename = 'aacr_scrape.json'

# Save the dictionary to a JSON file
with open(filename, 'w') as file:
    json.dump(all_sessions_data, file, indent=4)

In [33]:
all_sessions_data

{170: {'Session Title': 'MS.TEST.01 - AACR Annual Meeting 2024: Oral Presentations',
  'Text': '',
  'Presentations': []},
 465: {'Session Title': 'MW19 - Analytic Resources and Applications for AACR Project GENIE BPC Clinico-genomic Data',
  'Text': 'This session will provide an overview of AACR Project GENIE BPC and will feature a hands-on workshop to introduce learners to the {genieBPC} R package, which streamlines the processing of these rich clinico-genomic data. The session will close with a presentation on a novel statistical methodology for pan-cancer mutational signature analysis using the GENIE BPC data.',
  'Presentations': [{'Time': '8:00 AM - 8:01 AM',
    'Presenter': 'Katherine Panageas',
    'Affiliation': 'Memorial Sloan Kettering Cancer Center, New York, NY'},
   {'Time': '8:00 AM - 8:01 AM',
    'Presenter': 'Katherine Panageas',
    'Affiliation': 'Memorial Sloan Kettering Cancer Center, New York, NY'},
   {'Time': '8:01 AM - 8:10 AM',
    'Presenter': 'Alexander Pa

In [22]:
#len(all_sessions_data

{170: {'Session Title': 'MS.TEST.01 - AACR Annual Meeting 2024: Oral Presentations',
  'Text': '',
  'Presentations': []}}

In [17]:
#all_sessions_data_pass1 = all_sessions_data.copy()

In [19]:
#all_sessions_data_pass1

In [24]:
#all_sessions_data_pass1

In [28]:
all_sessions_data

{170: {'Session Title': 'MS.TEST.01 - AACR Annual Meeting 2024: Oral Presentations',
  'Text': '',
  'Presentations': []},
 465: {'Session Title': 'MW19 - Analytic Resources and Applications for AACR Project GENIE BPC Clinico-genomic Data',
  'Text': 'This session will provide an overview of AACR Project GENIE BPC and will feature a hands-on workshop to introduce learners to the {genieBPC} R package, which streamlines the processing of these rich clinico-genomic data. The session will close with a presentation on a novel statistical methodology for pan-cancer mutational signature analysis using the GENIE BPC data.',
  'Presentations': [{'Time': '8:00 AM - 8:01 AM',
    'Presenter': 'Katherine Panageas',
    'Affiliation': 'Memorial Sloan Kettering Cancer Center, New York, NY'},
   {'Time': '8:00 AM - 8:01 AM',
    'Presenter': 'Katherine Panageas',
    'Affiliation': 'Memorial Sloan Kettering Cancer Center, New York, NY'},
   {'Time': '8:01 AM - 8:10 AM',
    'Presenter': 'Alexander Pa

In [29]:
len(all_sessions_data)

26

In [31]:
#data